<a href="https://colab.research.google.com/github/0xbageltoes/dcm-colabs/blob/main/VisualChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download visual-chatgpt


In [ ]:
!rm -rf visual-chatgpt && git clone https://github.com/microsoft/visual-chatgpt

Cloning into 'visual-chatgpt'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 227 (delta 116), reused 177 (delta 89), pack-reused 0
Receiving objects: 100% (227/227), 10.65 MiB | 17.37 MiB/s, done.
Resolving deltas: 100% (116/116), done.


Go to directory


In [ ]:
cd visual-chatgpt/

/content/visual-chatgpt


Install packages


In [ ]:
!pip install -r requirements.txt

Set your OPENAI_API_KEY


In [ ]:
%env OPENAI_API_KEY=your_key

Import dependencies

In [ ]:
from visual_chatgpt import *

Run VisualChatGPT

In [ ]:
# load = "ImageCaptioning_cuda:0,Text2Image_cuda:0"
load = "Text2Image_cuda:0,ImageCaptioning_cuda:0,VisualQuestionAnswering_cuda:0,Image2Canny_cpu,Image2Line_cpu,Image2Pose_cpu,Image2Depth_cpu,CannyText2Image_cuda:0,InstructPix2Pix_cuda:0,Image2Seg_cuda:0"

load_dict = {e.split('_')[0].strip(): e.split('_')[1].strip() for e in load.split(',')}
bot = ConversationBot(load_dict=load_dict)
with gr.Blocks(css="#chatbot .overflow-y-auto{height:500px}") as demo:
    lang = gr.Radio(choices = ['Chinese','English'], value=None, label='Language')
    chatbot = gr.Chatbot(elem_id="chatbot", label="Visual ChatGPT")
    state = gr.State([])
    with gr.Row(visible=False) as input_raws:
        with gr.Column(scale=0.7):
            txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter, or upload an image").style(
                container=False)
        with gr.Column(scale=0.15, min_width=0):
            clear = gr.Button("Clear️")
        with gr.Column(scale=0.15, min_width=0):
            btn = gr.UploadButton("🖼️", file_types=["image"])
    lang.change(bot.init_agent, [lang], [input_raws, lang, txt, clear])
    txt.submit(bot.run_text, [txt, state], [chatbot, state])
    txt.submit(lambda: "", None, txt)
    btn.upload(bot.run_image, [btn, state, txt, lang], [chatbot, state, txt])
    clear.click(bot.memory.clear)
    clear.click(lambda: [], None, chatbot)
    clear.click(lambda: [], None, state)
    demo.launch(server_name="0.0.0.0", server_port=7861, share=True, debug=True)

In [ ]:
!ps -aux | grep "python"
# !kill -9 7507    